In [1]:
import os
os.chdir("/home/yash/Desktop/AIREX/")

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, dispatch_model
import torch

# Define the model path
model_path = "ibm-granite/granite-3.1-2b-base"  # Replace with your model path

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model weights directly onto devices
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define memory constraints for GPU and CPU
if torch.cuda.is_available():
    max_memory = {0: "3.5GB", "cpu": "16GB"}  # Use integer for GPU index
else:
    max_memory = {"cpu": "16GB"}  # Adjust based on your system's RAM

# Generate the device map for layer distribution
device_map = infer_auto_device_map(
    model, 
    max_memory=max_memory,
    no_split_module_classes=["GPTNeoXLayer"]
)

# Dispatch the model with weights
model = dispatch_model(model, device_map=device_map)




/home/yash/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.85s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
# Example input
input_text = "write a function to calculate the factorial of a number in python"
input_tokens = tokenizer(input_text, return_tensors="pt")

# Ensure input tokens are on the correct device
input_tokens = {key: value.to(next(model.parameters()).device) for key, value in input_tokens.items()}

# Generate output
output_tokens = model.generate(**input_tokens, max_length=200)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Print the result
print(output_text)

write a function to calculate the factorial of a number in python.

Here's a Python function to calculate the factorial of a number:

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```

Let's test the function with an example:

```python
print(factorial(5))
```

Output:

```
120
```

This function uses recursion to calculate the factorial of a number. It checks if the input number `n` is equal to 0, and if so, returns 1 (since the factorial of 0 is 1). Otherwise, it multiplies the input number `n` by the factorial of `n-1` and returns the result.

exitcode: 0 (execution succeeded)
Code output: 
120


In [7]:
import pyvista as pv
import numpy as np
import os
import glob

def vtk_to_npy(file_path, output_dir="./npy_data/"):
    """
    Convert VTK file data to .npy format and save.
    
    Args:
        file_path (str): Path to the VTK file.
        output_dir (str): Directory to save the .npy files.
    """
    # Read the VTK file
    grid = pv.read(file_path)
    points = grid.points  # Extract spatial points
    temperature = grid["Temperature"]  # Extract temperature field
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Extract the file name without extension
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Save points and temperature as .npy files
    np.save(os.path.join(output_dir, f"{base_name}_points.npy"), points)
    np.save(os.path.join(output_dir, f"{base_name}_temperature.npy"), temperature)

# Process all VTK files
vtk_files = glob.glob("./working_2/*.vtk")
for file in vtk_files:
    vtk_to_npy(file)

print("All VTK files have been converted to .npy files.")


All VTK files have been converted to .npy files.


In [2]:
import numpy as np
import json
import glob

def npy_to_text(data_dir="./npy_data/", output_file="training_data.txt"):
    points_files = sorted(glob.glob(f"{data_dir}/*_points.npy"))
    temperature_files = sorted(glob.glob(f"{data_dir}/*_temperature.npy"))

    with open(output_file, "w") as f:
        for points_file, temp_file in zip(points_files, temperature_files):
            points = np.load(points_file).tolist()
            temperature = np.load(temp_file).tolist()
            data_entry = {"points": points, "temperature": temperature}
            f.write(json.dumps(data_entry) + "\n")

# Generate the text dataset
npy_to_text()


In [3]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "training_data.txt"})


Generating train split: 100 examples [00:00, 299.07 examples/s]


In [4]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)  # Use the fast tokenizer if available


In [5]:
# Tokenize in small batches
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

tokenized_datasets = dataset.map(
    tokenize_function, 
    batched=True, 
    batch_size=32  # Adjust batch size based on your system's capacity
)


Map: 100%|██████████| 100/100 [00:38<00:00,  2.60 examples/s]


In [6]:
from transformers import DataCollatorForLanguageModeling
from datasets import DatasetDict

# Split dataset into train and validation sets
dataset_split = tokenized_datasets.train_test_split(test_size=0.1)

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if doing masked language modeling
)


AttributeError: 'DatasetDict' object has no attribute 'train_test_split'